In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import supertrend_lib as stlib
import sbeta_lib as sbeta
import smacross_lib as smacross
import CAPM_lib as capm
import LR_run_model_lib as LR_run
import LR_model_lib as LR_models

In [4]:
def add_update(eval_df,values):
    """ Add a a row to dataframe from a list of data points in the same sequence as the columns

    Args:
        eval_df (_type_): DataFrame receiving the new row
        values_list (_type_): A list of values in the same data types and sequance order as the dataframe columns

    Returns:
        _type_: The updated dataframe
    """
    tmp_df = pd.DataFrame([values], columns=eval_df.columns)
    eval_df = pd.concat([eval_df, tmp_df], ignore_index=True)
    return eval_df

In [3]:
def read_stocklist(filename):
    with open(filename) as f:
        stocklist = f.read()
    return stocklist.strip().split(",")


In [2]:
def update_models(stock_list):
    ret = LR_models.update_list_of_models(stock_list=stock_list)
    models_df = pd.DataFrame({'Stock' : stock_list,
                              'Model' : [f"Model{i}" for i in ret]})
    # print(models_df)
    return models_df

In [5]:
def run_supertrend(stock,start_date):
    return stlib.supertrend(stock,start_date=start_date)


In [6]:
def init_eval_table():
    columns = ['Last_Run','Stock','Last_Price','Supertrend_Winner','In_Supertrend_BUY','ST_Signal_Date', 
            'LR_Next_Day_Recomm', 'SMA_Crossed_Up','SMA_X_Date','SMA_FastXSlow', 'Beta', 'CAPM', 'Daily_VaR']

    dtypes = [np.datetime64, str, float, bool, str, str, np.datetime64, str, str, float, float, str]

    # Initialize the DataFrame with empty rows
    eval_df = pd.DataFrame(columns=columns)

    # Convert the column data types
    for col, dtype in zip(columns, dtypes):
        eval_df[col] = eval_df[col].astype(dtype)
        
    return eval_df

In [7]:
def first_date_N_years_ago(years):
    # Calculate start and end dates N years back
    end_date = datetime.date.today() - datetime.timedelta(days=1)
    start_date = end_date.replace(year=end_date.year - years, month=1, day=1)
    return start_date.strftime("%Y-%m-%d")


In [8]:
def load_sp500_list():
    stocks_list_csv = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp_df = pd.DataFrame()
    
    try:
        wiki_data=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies') # Open the link and download S&P company details in a table
        data = wiki_data[0] # All data is stored in first cell
        sp_df = data.sort_values(by=['Symbol'], ascending=True) # Sort the dataframe on ticker in alphabetical ascending order
    except:
        print("Cannot open file", stocks_list_csv)

    # remove the dotted symbols, they are redundant 
    no_dot_symbols = [i for i in sp_df['Symbol'] if i.find('.')==-1]
    sp_df = sp_df[sp_df['Symbol'].isin(no_dot_symbols)]
    
    # returns count_row, count_col, df
    return sp_df.shape[0], sp_df.shape[1], sp_df


In [21]:
m,n,sp_df = load_sp500_list()
# sp_df[sp_df['Symbol'].isin([i for i in sp_df['Symbol'] if i.find('.')>0])]

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded


In [9]:
def recommendation_table(eval_df,stock_list, lookback_years=1, sma_fast=40, sma_slow=200):
    stock_list = [str(s).upper() for s in stock_list]
    print("Stock List:",stock_list)
    start_date = first_date_N_years_ago(lookback_years)
    today = datetime.datetime.today().strftime("%Y-%m-%d")

    for s in stock_list:
        winner,buysell,buysell_date,close_price,stock_data = run_supertrend(s,start_date)
        print(f"{s}",end=',')
        sma_sig, sma_date, fastXslow = smacross.sma_xing(stock_data,sma_fast,sma_slow)
        beta, market_data = sbeta.get_beta(stock_data)
        CAPM, VaR = capm.CAPM_VaR(stock_data=stock_data,market_data=market_data,bond_mat_duration = lookback_years,stock_beta=beta)
        LR_recommend = LR_run.get_recommendation(stock=s,lookback=lookback_years)
        LR_recommend_str = f"{LR_recommend[0]},{LR_recommend[1]},{LR_recommend[2]}"
        new_row = [today, s ,close_price, winner, buysell,buysell_date , LR_recommend_str, 
                sma_sig, sma_date, fastXslow,
                beta, CAPM,VaR]
        eval_df = add_update(eval_df=eval_df,values = new_row)
        
    return eval_df

In [10]:
def eval_all_sp500(lookback_years = 1,sma_fast = 40, sma_slow = 200 ):
    n,m,sp_df = load_sp500_list()
    stocks_sp500 = sp_df.Symbol.values
    stocks_sp500 = [s.strip().upper() for s in list(stocks_sp500)]

    eval_df = init_eval_table()
    eval_df = recommendation_table(eval_df,
                                stock_list=stocks_sp500, 
                                lookback_years=lookback_years, 
                                sma_fast=sma_fast, 
                                sma_slow=sma_slow)
    
    return eval_df

In [11]:
def eval_list_from_file(filename='./stocks_list.txt',lookback_years = 1,sma_fast = 40, sma_slow = 200 ):
    stocks_from_file = read_stocklist(filename=filename)
    stocks_from_file = [s.strip().upper() for s in list(stocks_from_file)]
    today = datetime.datetime.today().strftime("%Y-%m-%d")

    eval_df = init_eval_table()
    eval_df = recommendation_table(eval_df,
                                stock_list=stocks_from_file, 
                                lookback_years=lookback_years, 
                                sma_fast=sma_fast, 
                                sma_slow=sma_slow)
    
    return eval_df

In [12]:
def screen_for_buys(eval_df):
        buys_df = eval_df[ (eval_df['Supertrend_Winner']==True) &  
                (eval_df['In_Supertrend_BUY']=='Buy') & 
                (eval_df['LR_Next_Day_Recomm'] == 'Buy,Buy,Buy') &
                (eval_df['SMA_Crossed_Up']=='Buy')].sort_values(by=['Supertrend_Winner','In_Supertrend_BUY','ST_Signal_Date','SMA_Crossed_Up','SMA_X_Date'],
                                                                ascending=[False,True,False,True,False])
        
        return buys_df

### Evaluate Stocks List

In [ ]:
stock_list = read_stocklist(filename='./stocks_list.txt')
ret = update_models(stock_list=stock_list)
ret

In [26]:
eval_df = eval_df_from_file = eval_list_from_file(filename='stocks_list.txt',lookback_years=2)
# eval_df_sp500 = eval_all_sp500(lookback_years=1)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [27]:
eval_df

,Last_Run,Stock,Last_Price,Supertrend_Winner,In_Supertrend_BUY,ST_Signal_Date,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
0,2023-03-25,QQQ,310.89,False,Buy,2023-03-21,"Buy,Buy,Buy",Buy,2023-02-28,"(40, 200)",1.2719,3.14,-3.87
1,2023-03-25,AAPL,160.25,False,Buy,2023-01-23,"Buy,Buy,Buy",Buy,2023-03-13,"(40, 200)",1.2957,3.13,-4.56
2,2023-03-25,MSFT,280.57,False,Buy,2023-03-16,"Sell,Sell,Sell",Buy,2023-03-09,"(40, 200)",1.2551,3.15,-4.42
3,2023-03-25,NVDA,267.79,True,Buy,2023-01-12,"Buy,Buy,Buy",Buy,2023-01-24,"(40, 200)",2.1728,2.71,-8.51
4,2023-03-25,META,206.01,False,Buy,2022-11-10,"Buy,Buy,Buy",Buy,2023-02-23,"(40, 200)",1.6742,2.95,-7.93
5,2023-03-25,IBM,125.29,False,Sell,2022-12-15,"Sell,Buy,Sell",Sell,2023-03-16,"(40, 200)",0.5276,3.50,-3.46
6,2023-03-25,QCOM,124.76,False,Sell,2023-02-21,"Buy,Buy,Buy",Buy,2023-03-06,"(40, 200)",1.5291,3.02,-6.04
7,2023-03-25,MU,61.16,False,Buy,2023-03-23,"Buy,Buy,Buy",Buy,2023-02-24,"(40, 200)",1.5183,3.02,-6.47
8,2023-03-25,TSLA,190.41,False,Sell,2023-03-09,"Sell,Sell,Sell",Sell,2022-10-03,"(40, 200)",1.8310,2.87,-9.42
9,2023-03-25,SPY,395.75,False,Sell,2023-03-01,"Buy,Buy,Buy",Buy,2023-02-01,"(40, 200)",1.0002,3.28,-2.87


In [42]:
buys_eval_df= screen_for_buys(eval_df=eval_df)
buys_eval_df

,Last_Run,Stock,Last_Price,Supertrend_Winner,In_Supertrend_BUY,ST_Signal_Date,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
19,2023-03-25,ADBE,374.96,True,Buy,2023-03-17,"Buy,Buy,Buy",Buy,2023-03-16,"(40, 200)",1.4486,3.06,-5.70
18,2023-03-25,CDNS,204.48,True,Buy,2023-01-17,"Buy,Buy,Buy",Buy,2022-12-19,"(40, 200)",1.3805,3.09,-5.56
3,2023-03-25,NVDA,267.79,True,Buy,2023-01-12,"Buy,Buy,Buy",Buy,2023-01-24,"(40, 200)",2.1728,2.71,-8.51


In [29]:
eval_df.to_csv("./stocks_buys_eval_df_table.csv")

In [30]:
buys_safe = buys_eval_df[(buys_eval_df['Beta']>1) & (buys_eval_df['Beta']<2.0) ].sort_values('Daily_VaR',ascending=False)

In [31]:
len(buys_safe)

3

In [32]:
buys_safe

,Last_Run,Stock,Last_Price,Supertrend_Winner,In_Supertrend_BUY,ST_Signal_Date,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
16,2023-03-25,AVGO,636.17,True,Buy,2022-11-08,"Sell,Sell,Sell",Buy,2022-12-29,"(40, 200)",1.3336,3.11,-4.95
18,2023-03-25,CDNS,204.48,True,Buy,2023-01-17,"Buy,Buy,Buy",Buy,2022-12-19,"(40, 200)",1.3805,3.09,-5.56
19,2023-03-25,ADBE,374.96,True,Buy,2023-03-17,"Buy,Buy,Buy",Buy,2023-03-16,"(40, 200)",1.4486,3.06,-5.70


In [33]:
buys_safe.to_csv("./top_picks.csv")

In [46]:
LR_Next_Day_Recomm_only = eval_df[eval_df['LR_Next_Day_Recomm']=='Buy,Buy,Buy'].sort_values('Daily_VaR',ascending=False)
LR_Next_Day_Recomm_only

,Last_Run,Stock,Last_Price,Supertrend_Winner,In_Supertrend_BUY,ST_Signal_Date,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
9,2023-03-25,SPY,395.75,False,Sell,2023-03-01,"Buy,Buy,Buy",Buy,2023-02-01,"(40, 200)",1.0002,3.28,-2.87
0,2023-03-25,QQQ,310.89,False,Buy,2023-03-21,"Buy,Buy,Buy",Buy,2023-02-28,"(40, 200)",1.2719,3.14,-3.87
1,2023-03-25,AAPL,160.25,False,Buy,2023-01-23,"Buy,Buy,Buy",Buy,2023-03-13,"(40, 200)",1.2957,3.13,-4.56
17,2023-03-25,DE,386.50,True,Sell,2023-01-19,"Buy,Buy,Buy",Buy,2022-11-01,"(40, 200)",0.8231,3.36,-4.96
18,2023-03-25,CDNS,204.48,True,Buy,2023-01-17,"Buy,Buy,Buy",Buy,2022-12-19,"(40, 200)",1.3805,3.09,-5.56
19,2023-03-25,ADBE,374.96,True,Buy,2023-03-17,"Buy,Buy,Buy",Buy,2023-03-16,"(40, 200)",1.4486,3.06,-5.70
11,2023-03-25,AMZN,98.13,False,Sell,2023-02-09,"Buy,Buy,Buy",Sell,2022-01-19,"(40, 200)",1.5189,3.02,-5.82
6,2023-03-25,QCOM,124.76,False,Sell,2023-02-21,"Buy,Buy,Buy",Buy,2023-03-06,"(40, 200)",1.5291,3.02,-6.04
7,2023-03-25,MU,61.16,False,Buy,2023-03-23,"Buy,Buy,Buy",Buy,2023-02-24,"(40, 200)",1.5183,3.02,-6.47
4,2023-03-25,META,206.01,False,Buy,2022-11-10,"Buy,Buy,Buy",Buy,2023-02-23,"(40, 200)",1.6742,2.95,-7.93


In [40]:
single_stock_df = init_eval_table()
single_stock_df = recommendation_table(single_stock_df,['msft'], lookback_years=5, sma_fast=40, sma_slow=200)
single_stock_df


Stock List: ['MSFT']
[*********************100%***********************]  1 of 1 completed


,Last_Run,Stock,Last_Price,Supertrend_Winner,In_Supertrend_BUY,ST_Signal_Date,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
0,2023-03-25,MSFT,280.57,False,Buy,2023-03-16,"Sell,Sell,Sell",Buy,2023-03-09,"(40, 200)",1.215,10.53,-4.73
